In [1]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE


warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import json

In [75]:
##Config preparation

with open('../config/config.json', 'r') as f:
    config = json.load(f)


In [76]:
path = config.get('rawpath')+config.get('corpus')+'/'+'corpus.csv'

In [77]:
path

'../data/raw/20newsgroup/corpus.csv'

In [ ]:
corpus = datatable.fread(path_corpus)

# Spacy load

In [87]:
import spacy
import time 
from spacy_langdetect import LanguageDetector
nlp = spacy.load("en_core_web_md")

In [ ]:
corpus.head()

In [ ]:

clase_positiva = corpus[datatable.f.target==1,:]
clase_positiva.shape

In [ ]:
clase_negativa = corpus[datatable.f.target==0,:]
clase_negativa.shape

In [ ]:
indices_originales = corpus[:,1:4]

In [ ]:
indices_originales

In [ ]:
indices_originales.to_csv('corpus_df/indices_originales.csv')

In [ ]:
filas_seleccionadas = random.sample(range(1,clase_positiva.shape[0]),300000)
filas_totales = range(1,clase_positiva.shape[0])
filas_para_clase_negativa=set(filas_totales)-set(filas_seleccionadas)

In [ ]:
len(filas_para_clase_negativa)

In [ ]:
#X = corpus.to_pandas()['text']
#y = corpus.to_pandas()['target']

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(
#        X, y,stratify=y, test_size=0.4)

#X_test, X_val, y_test, y_val = train_test_split(
#        X_test, y_test, stratify=y_test, test_size=0.5)

In [ ]:
X_train.shape

In [ ]:
y_train.unique()

In [ ]:
y_val.shape

In [ ]:
clase_positiva_final = clase_positiva[filas,:]

In [ ]:
clase_positiva_final.shape

In [ ]:
clase_positiva_enneg = clase_positiva[list(filas_para_clase_negativa),:]

In [ ]:
clase_positiva_enneg.shape

In [ ]:
clase_positiva_enneg.rbind(clase_negativa)

In [ ]:
clase_positiva_enneg['target']=0

In [ ]:
new_corpus=clase_positiva.rbind(clase_positiva_enneg)

In [ ]:
clase_positiva.to_csv('corpus_df/corpus_procesado.csv')

In [ ]:
indices = clase_positiva[:,1:4]

In [ ]:
indices.to_csv('corpus_df/indices.csv')

## Preprocesado

In [ ]:
corpus = datatable.fread('corpus_df/corpus_procesado.csv')

In [ ]:
corpus = corpus.to_pandas()

In [ ]:
docs = corpus['text'].to_list()

In [ ]:
len(docs)

In [ ]:
def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop)
filtered_tokens = []
i = time.time()
cont = 0
for doc in nlp.pipe(docs,disable=["parser","tagger"]):
    tokens = " ".join(token.lemma_ for token in doc if token_filter(token))
    filtered_tokens.append(tokens)
f=time.time()
elapsed = f - i

print ('han pasado {0} s'.format(elapsed))

In [ ]:
corpus['normalizado'] =filtered_tokens

In [ ]:
corpusdt = datatable.Frame(corpus)

In [ ]:
corpusdt['long']=corpusdt[:, datatable.f.normalizado.len()]

In [ ]:
del corpusdt[datatable.f.long<=10, :]
del corpusdt [:,datatable.f.text]
del corpusdt [:,datatable.f.long]

In [ ]:
corpusdt = corpusdt.to_pandas()

In [ ]:
textos = corpusdt['normalizado'].to_list()

In [ ]:
langs = []
i = time.time()
cont = 0
for doc in nlp.pipe(textos,disable=["parser","tagger"]):
    langs.append(doc.lang_)
f = time.time()

In [ ]:
elapsed = f - i
corpusdt['lang']=langs

In [ ]:
corpusdt.head()

In [ ]:
corpusdt2 = datatable.Frame(corpusdt)
corpusdt2.shape

In [ ]:
del corpusdt2[datatable.f.lang!='en',:]

In [ ]:
corpusdt2.to_csv('corpus_df/normalizado.csv')

## Generación de corpuses


### 50000 documentos 

In [ ]:
indice_original = pd.read_csv('corpus_df/indices_originales.csv')

In [ ]:
indice_original['target'].sum()

In [ ]:
corpus = datatable.fread('corpus_df/normalizado.csv')
corpus = corpus.to_pandas()

In [ ]:
corpus.shape

In [ ]:
corpus['target']=corpus['target'].apply(lambda x: int(x==True) ) 

In [ ]:
corpus['target'].sum()

In [ ]:
corpus = corpus.merge(indice_original,how='left',on='index',suffixes=['','_original'])
corpus.dropna()

In [ ]:
corpus.loc[(corpus.target==0) &(corpus.target_original==1) & (corpus.index>500010),'target_original']=0

In [ ]:
corpus.target_original.sum() #breve dufeebcua

In [ ]:
indice_original[indice_original.target==1]

In [ ]:
corpus.to_csv('corpus_tm/corpus_normalizado_t_original.csv')

In [ ]:
corpus = pd.read_csv('corpus_tm/corpus_normalizado_t_original.csv')

### Particionamiento

Ahora vamos a proceder a separar el corpus original en 3:
+ 50.000 para topic modeling solo con clase positiva. En este caso solo guardaremos los índices
+ 3 corpus de 50.000, 100.000 y 250.000 documentos mezclando clase positiva y negativa.


## Guardo indices con corpus solo target positiva

In [ ]:
np.random.seed(101)
df_positivo = corpus.loc[(corpus.target_original==1) & (corpus.target==1),:]
indices_positivo = df_positivo['index']

In [ ]:
indices_smp = df_positivo.sample(n=50000).loc[:,'index']

In [ ]:
indices_smp.to_csv('corpus_tm/indices_positivos_50k.csv')

### Guardo los indices negativos


In [ ]:
df_negativo = corpus.loc[(corpus.target_original==0) & (corpus.target==0),:]

## Genero subconjuntos de 25 mil positivos y negativos

In [ ]:
np.random.seed(123)
indices_smp_pos0 = df_positivo.sample(n=25000).loc[:,'index']
indices_smp_neg0 = df_negativo.sample(n=25000).loc[:,'index']
indices0 = indices_smp_pos0 + indices_smp_neg0
indices0.to_csv('corpus_tm/corpus1_50k.csv')

## Generamos subconjuntos de 50.000 positivo y negativo

In [ ]:
np.random.seed(13)
indices_smp_pos1 = df_positivo.sample(n=50000).loc[:,'index']
indices_smp_neg1 = df_negativo.sample(n=50000).loc[:,'index']
indices1 = indices_smp_pos1 + indices_smp_neg1
indices1.to_csv('corpus_tm/corpus1_100k.csv')

## Generamos subconjuntos de 125.000 positivo y negativo

In [ ]:
np.random.seed(7)
indices_smp_pos1 = df_positivo.sample(n=125000).loc[:,'index']
indices_smp_neg1 = df_negativo.sample(n=125000).loc[:,'index']
indices1 = indices_smp_pos1 + indices_smp_neg1
indices1.to_csv('corpus_tm/corpus1_250k.csv')

In [ ]:
corpus.head()

In [ ]:

clase_positiva = corpus[datatable.f.target==1,:]
clase_positiva.shape

In [ ]:
clase_negativa = corpus[datatable.f.target==0,:]
clase_negativa.shape

In [ ]:
indices_originales = corpus[:,1:4]

In [ ]:
indices_originales

In [ ]:
indices_originales.to_csv('corpus_df/indices_originales.csv')

In [ ]:
filas_seleccionadas = random.sample(range(1,clase_positiva.shape[0]),300000)
filas_totales = range(1,clase_positiva.shape[0])
filas_para_clase_negativa=set(filas_totales)-set(filas_seleccionadas)

In [ ]:
len(filas_para_clase_negativa)

In [ ]:
#X = corpus.to_pandas()['text']
#y = corpus.to_pandas()['target']

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(
#        X, y,stratify=y, test_size=0.4)

#X_test, X_val, y_test, y_val = train_test_split(
#        X_test, y_test, stratify=y_test, test_size=0.5)

In [ ]:
X_train.shape

In [ ]:
y_train.unique()

In [ ]:
y_val.shape

In [ ]:
clase_positiva_final = clase_positiva[filas,:]

In [ ]:
clase_positiva_final.shape

In [ ]:
clase_positiva_enneg = clase_positiva[list(filas_para_clase_negativa),:]

In [ ]:
clase_positiva_enneg.shape

In [ ]:
clase_positiva_enneg.rbind(clase_negativa)

In [ ]:
clase_positiva_enneg['target']=0

In [ ]:
new_corpus=clase_positiva.rbind(clase_positiva_enneg)

In [ ]:
clase_positiva.to_csv('corpus_df/corpus_procesado.csv')

In [ ]:
indices = clase_positiva[:,1:4]

In [ ]:
indices.to_csv('corpus_df/indices.csv')

## Preprocesado

In [ ]:
corpus = datatable.fread('corpus_df/corpus_procesado.csv')

In [ ]:
corpus = corpus.to_pandas()

In [ ]:
docs = corpus['text'].to_list()

In [ ]:
len(docs)

In [ ]:
def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop)
filtered_tokens = []
i = time.time()
cont = 0
for doc in nlp.pipe(docs,disable=["parser","tagger"]):
    tokens = " ".join(token.lemma_ for token in doc if token_filter(token))
    filtered_tokens.append(tokens)
f=time.time()
elapsed = f - i

print ('han pasado {0} s'.format(elapsed))

In [ ]:
corpus['normalizado'] =filtered_tokens

In [ ]:
corpusdt = datatable.Frame(corpus)

In [ ]:
corpusdt['long']=corpusdt[:, datatable.f.normalizado.len()]

In [ ]:
del corpusdt[datatable.f.long<=10, :]
del corpusdt [:,datatable.f.text]
del corpusdt [:,datatable.f.long]

In [ ]:
corpusdt = corpusdt.to_pandas()

In [ ]:
textos = corpusdt['normalizado'].to_list()

In [ ]:
langs = []
i = time.time()
cont = 0
for doc in nlp.pipe(textos,disable=["parser","tagger"]):
    langs.append(doc.lang_)
f = time.time()

In [ ]:
elapsed = f - i
corpusdt['lang']=langs

In [ ]:
corpusdt.head()

In [ ]:
corpusdt2 = datatable.Frame(corpusdt)
corpusdt2.shape

In [ ]:
del corpusdt2[datatable.f.lang!='en',:]

In [ ]:
corpusdt2.to_csv('corpus_df/normalizado.csv')

## Generación de corpuses


### 50000 documentos 

In [ ]:
indice_original = pd.read_csv('corpus_df/indices_originales.csv')

In [ ]:
indice_original['target'].sum()

In [ ]:
corpus = datatable.fread('corpus_df/normalizado.csv')
corpus = corpus.to_pandas()

In [ ]:
corpus.shape

In [ ]:
corpus['target']=corpus['target'].apply(lambda x: int(x==True) ) 

In [ ]:
corpus['target'].sum()

In [ ]:
corpus = corpus.merge(indice_original,how='left',on='index',suffixes=['','_original'])
corpus.dropna()

In [ ]:
corpus.loc[(corpus.target==0) &(corpus.target_original==1) & (corpus.index>500010),'target_original']=0

In [ ]:
corpus.target_original.sum() #breve dufeebcua

In [ ]:
indice_original[indice_original.target==1]

In [ ]:
corpus.to_csv('corpus_tm/corpus_normalizado_t_original.csv')

In [ ]:
corpus = pd.read_csv('corpus_tm/corpus_normalizado_t_original.csv')

### Particionamiento

Ahora vamos a proceder a separar el corpus original en 3:
+ 50.000 para topic modeling solo con clase positiva. En este caso solo guardaremos los índices
+ 3 corpus de 50.000, 100.000 y 250.000 documentos mezclando clase positiva y negativa.


## Guardo indices con corpus solo target positiva

In [ ]:
np.random.seed(101)
df_positivo = corpus.loc[(corpus.target_original==1) & (corpus.target==1),:]
indices_positivo = df_positivo['index']

In [ ]:
indices_smp = df_positivo.sample(n=50000).loc[:,'index']

In [ ]:
indices_smp.to_csv('corpus_tm/indices_positivos_50k.csv')

### Guardo los indices negativos


In [ ]:
df_negativo = corpus.loc[(corpus.target_original==0) & (corpus.target==0),:]

## Genero subconjuntos de 25 mil positivos y negativos

In [ ]:
np.random.seed(123)
indices_smp_pos0 = df_positivo.sample(n=25000).loc[:,'index']
indices_smp_neg0 = df_negativo.sample(n=25000).loc[:,'index']
indices0 = indices_smp_pos0 + indices_smp_neg0
indices0.to_csv('corpus_tm/corpus1_50k.csv')

## Generamos subconjuntos de 50.000 positivo y negativo

In [ ]:
np.random.seed(13)
indices_smp_pos1 = df_positivo.sample(n=50000).loc[:,'index']
indices_smp_neg1 = df_negativo.sample(n=50000).loc[:,'index']
indices1 = indices_smp_pos1 + indices_smp_neg1
indices1.to_csv('corpus_tm/corpus1_100k.csv')

## Generamos subconjuntos de 125.000 positivo y negativo

In [ ]:
np.random.seed(7)
indices_smp_pos1 = df_positivo.sample(n=125000).loc[:,'index']
indices_smp_neg1 = df_negativo.sample(n=125000).loc[:,'index']
indices1 = indices_smp_pos1 + indices_smp_neg1
indices1.to_csv('corpus_tm/corpus1_250k.csv')

# 20 Newsgroups

In [4]:
from sklearn.datasets import fetch_20newsgroups

In [5]:
data = fetch_20newsgroups(subset="all",)

In [6]:
data.data[0]

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [16]:
df = pd.DataFrame()

In [18]:
df = df.assign(text=data["data"]).assign(target=data["target"])

In [21]:
data['target_names'][3]

'comp.sys.ibm.pc.hardware'

In [10]:
df

,text,target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4
...,...,...
18841,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13
18842,From: rdell@cbnewsf.cb.att.com (richard.b.dell...,12
18843,From: westes@netcom.com (Will Estes)\nSubject:...,3
18844,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1


In [68]:
df.text=df.text.replace(to_replace='From:(.*\n)',value='',regex=True)

In [69]:
df.text = df.text.replace(to_replace='lines:(.*\n)',value=' ',regex=True)
df.text =df.text.replace(to_replace='Subject:(.*\n)',value='',regex=True)
df.text =df.text.replace(to_replace='Re(.*\n)',value='',regex=True)

df.text =df.text.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df.text =df.text.replace(to_replace='-',value=' ',regex=True)
df.text =df.text.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df.text =df.text.replace(to_replace='  ',value='',regex=True)                #remove double white space
df.text = df.text.replace(to_replace='Subject ',value=' ',regex=True)

df.text =df.text.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace


In [70]:
df.head()

,text,target
0,Organization Post Office Carnegie Mellon Pitts...,10
1,Summary Seek recommendations for VLB video car...,3
2,Lines 95 Nntp Posting Host viktoria.dsv.su.se ...,17
3,Originator guyd pal500.austin.ibm.com Organiza...,3
4,Organization Sophomore Mechanical Engineering ...,4


In [71]:
path

NameError: name 'path' is not defined

In [79]:
df.to_csv(path,index=False)

In [82]:
import re
def remove_emails(text):
    regex =  r'\S*@\S*\s?'
    return re.sub(regex, '', text )

def remove_newlinechars(text):
    '''
    Substitute any newline chars with a whitespach
    
    The `regex` can be tried at: https://regex101.com/r/2fImPz/1/
    '''
    regex = r'\s+'
    return re.sub(regex, ' ', text)

def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop)
filtered_tokens = []
i = time.time()
cont = 0

In [83]:
import dask.dataframe as ddf
dask_dataframe = ddf.from_pandas(df, npartitions=6)

In [84]:
import time
t0 = time.time()
def clean_text(df):
    '''
    Take in a Dataframe, and process it
    '''
    df["cleaned_text"] = df.text.map(lambda text:text.lower()).map(remove_emails).map(remove_newlinechars)
    
    return df
df = clean_text(df)

In [85]:
docs = df['cleaned_text'].to_list()

In [102]:
filtered_tokens = []
for doc in nlp.pipe(docs,disable=["parser","tagger"],n_threads=10):
    tokens = " ".join(token.lemma_ for token in doc if token_filter(token))
    filtered_tokens.append(tokens)

t1 = time.time()
print("Time to process without Dask {}".format(t1-t0))

Time to process without Dask 2128.312903881073


In [107]:
import copy
corpus = copy.deepcopy(df)

In [104]:
len(filtered_tokens) == df.shape

False

In [108]:
corpus['cleaned_text']=filtered_tokens

In [106]:
del corpus['text']

In [109]:
corpus.to_csv(config.get('interimpath')+'/20newsgroup/corpus_normalizado.csv')

In [110]:
corpus = pd.read_csv(config.get('interimpath')+'/20newsgroup/corpus_normalizado.csv')

In [111]:
corpus = pd.read_csv('../data/interim/20newsgroup/corpus_normalizado.csv')

In [112]:
corpus.groupby(by='target').count()

,Unnamed: 0,text,cleaned_text
target,,,
0,799,799,799
1,973,973,973
2,985,985,985
3,982,982,982
4,963,963,963
5,988,988,988
6,975,975,975
7,990,990,990
8,996,996,996


In [113]:
corpus['Target']=corpus.target.apply(lambda x:  x==3)

In [114]:
del corpus ['Unnamed: 0']

In [115]:
corpus.to_csv('../data/interim/20newsgroup/corpus_normalizado.csv',index=False)

In [2]:
import pandas as pd

In [3]:
corpus = pd.read_csv('../data/interim/20newsgroup/corpus_normalizado.csv')

In [4]:
corpus.head()

,text,target,cleaned_text,Target
0,Organization Post Office Carnegie Mellon Pitts...,10,organization post office carnegie mellon pitts...,False
1,Summary Seek recommendations for VLB video car...,3,summary seek recommendation vlb video card nnt...,True
2,Lines 95 Nntp Posting Host viktoria.dsv.su.se ...,17,line 95 nntp post host viktoria.dsv.su.se orga...,False
3,Originator guyd pal500.austin.ibm.com Organiza...,3,originator guyd pal500.austin.ibm.com organiza...,True
4,Organization Sophomore Mechanical Engineering ...,4,organization sophomore mechanical engineer car...,False


# REUTERS

In [72]:
from nltk.corpus import reuters
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [82]:
categories = []
file_count = []

# count each tag's number of documents
for i in reuters.categories():
    """print("$ There are {} documents included in topic \"{}\""
          .format(len(reuters.fileids(i)), i))"""
    file_count.append(len(reuters.fileids(i)))
    categories.append(i)

# create a dataframe out of the counts
df = pd.DataFrame(
    {'categories': categories, "file_count": file_count}) \
    .sort_values('file_count', ascending=False)
print(df.head())

   categories  file_count
21       earn        3964
0         acq        2369
46   money-fx         717
26      grain         582
17      crude         578


In [91]:
reuters.abspath(reuters.fileids()[1])

ZipFilePathPointer('/home/rdelaguila/nltk_data/corpora/reuters.zip', 'reuters/test/14828')

In [99]:
fileids = reuters.fileids()

# Initialize empty lists to store categories and raw text
categories = []
text = []

# Loop through each file id and collect each files categories and raw text
for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

# Combine lists into pandas dataframe. reutersDf is the final dataframe. 
reutersDf = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})

In [113]:
reutersDf.to_csv(config.get('rawpath')+config.get('corpus')+'/corpus.csv')

In [114]:
reutersDf.head()

,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...


In [115]:
import dask.dataframe as ddf
dask_dataframe = ddf.from_pandas(reutersDf, npartitions=6)

In [123]:
import time
t0 = time.time()
def clean_text(df):
    '''
    Take in a Dataframe, and process it
    '''
    df["cleaned_text"] = df.text.map(lambda text:text.lower()).map(remove_emails).map(remove_newlinechars)
    
    return df
df = clean_text(reutersDf)

In [124]:
df

,ids,categories,text,cleaned_text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,asian exporters fear damage from u.s.-japan ri...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,china daily says vermin eat 7-12 pct grain sto...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,japan to revise long-term energy demand downwa...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,thai trade deficit widens in first quarter tha...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,indonesia sees cpo price rising sharply indone...
...,...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...,u.k. money market shortage forecast revised do...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...,knight-ridder inc &lt;krn> sets quarterly qtly...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...,technitrol inc &lt;tnl> sets quarterly qtly di...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...,nationwide cellular service inc &lt;ncel> 4th ...


In [129]:
docs = df['cleaned_text']
corpus=copy.deepcopy(df)
filtered_tokens=[]
for doc in nlp.pipe(docs,disable=["parser","tagger"]):
    tokens = " ".join(token.lemma_ for token in doc if token_filter(token))
    filtered_tokens.append(tokens)

corpus['texto_normalizado']=filtered_tokens

In [130]:
corpus.head()

,ids,categories,text,cleaned_text,texto_normalizado
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,asian exporters fear damage from u.s.-japan ri...,asian exporter fear damage u.s.-japan rift mou...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,china daily says vermin eat 7-12 pct grain sto...,china daily say vermin eat 7 12 pct grain stoc...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,japan to revise long-term energy demand downwa...,japan revise long term energy demand downwards...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,thai trade deficit widens in first quarter tha...,thai trade deficit widen quarter thailand trad...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,indonesia sees cpo price rising sharply indone...,indonesia see cpo price rise sharply indonesia...


In [131]:
del corpus['text']

In [132]:
corpus.to_csv(config.get('interimpath')+'/reuters/corpus_normalizado.csv')

In [1]:
# We are setting acq as positive class

In [4]:
corpus = pd.read_csv(config.get('interimpath')+'/reuters/corpus_normalizado.csv')

In [5]:
corpus.head()

,Unnamed: 0,ids,categories,cleaned_text,texto_normalizado
0,0,test/14826,['trade'],asian exporters fear damage from u.s.-japan ri...,asian exporter fear damage u.s.-japan rift mou...
1,1,test/14828,['grain'],china daily says vermin eat 7-12 pct grain sto...,china daily say vermin eat 7 12 pct grain stoc...
2,2,test/14829,"['crude', 'nat-gas']",japan to revise long-term energy demand downwa...,japan revise long term energy demand downwards...
3,3,test/14832,"['corn', 'grain', 'rice', 'rubber', 'sugar', '...",thai trade deficit widens in first quarter tha...,thai trade deficit widen quarter thailand trad...
4,4,test/14833,"['palm-oil', 'veg-oil']",indonesia sees cpo price rising sharply indone...,indonesia see cpo price rise sharply indonesia...


In [8]:
 corpus['categories'] 

0                                                ['trade']
1                                                ['grain']
2                                     ['crude', 'nat-gas']
3        ['corn', 'grain', 'rice', 'rubber', 'sugar', '...
4                                  ['palm-oil', 'veg-oil']
                               ...                        
10783                             ['interest', 'money-fx']
10784                                             ['earn']
10785                                             ['earn']
10786                                             ['earn']
10787                                             ['earn']
Name: categories, Length: 10788, dtype: object

In [14]:
corpus['Target']=corpus.categories.apply(lambda x:  'acq' in x)

In [15]:
corpus.Target.sum()

2369

In [17]:
del corpus ['Unnamed: 0']

In [18]:
corpus.to_csv(config.get('interimpath')+'/reuters/corpus_normalizado.csv',index=False)